# Audio MNIST
### RNN
#### 20213013 김어진


## 1. 데이터 분석

#### Audio MNIST 데이터셋 특징
- 목표: 음성 신호를 통해 0부터 9까지의 숫자를 인식하는 모델을 훈련.
- 형태: 각 숫자(0~9) 음성 파일은 WAV 형식으로 저장되어 있습니다. 각 음성 파일은 숫자를 발음한 오디오 클립.
- 샘플: 각 오디오 파일은 고유한 숫자 음성을 포함하고 있으며, 일반적으로 각 숫자에 대해 여러 샘플이 존재.

#### 구성 요소
- 샘플링 레이트: 일반적으로 음성 데이터는 일정한 샘플링 레이트. Audio MNIST의 경우, 보통 16kHz(16,000 샘플/초) 샘플링 레이트로 저장.
- 길이: 각 오디오 클립의 길이는 일정하지 않으며, 대체로 몇 백 밀리초 정도.
레이블: 각 음성 파일은 해당 숫자를 레이블(0~9)화한 음성.

#### waveform / mel-spectrogram
- waveform: 오디오 신호의 시간 영역 표현. 시간에 따른 음압의 변화를 시각화.
- mel-spectrogram: 주파수 영역 표현. 음성 신호를 주파수 대역으로 분해하고, 각 주파수 대역의 에너지 크기를 시각화.

#### 6가지 모델 구성
- Waveform + RNN
- Waveform + LSTM
- Waveform + GRU
- Mel-Spectrogram + RNN
- Mel-Spectrogram + LSTM
- Mel-Spectrogram + GRU

## 데이터 로드

In [1]:
# 데이터 불러오기
import torch
import matplotlib.pyplot as plt

# audio_mnist.pt 파일 로드
file_path = r"C:\Users\HOME\Downloads\audio_mnist.pt"  # 파일 경로 설정
audio_data = torch.load(file_path)

AttributeError: module 'torch' has no attribute 'load'

In [ ]:
# waveform 시각화
# 데이터 구조 확인
print(type(audio_data))  # 데이터 타입 확인
print(audio_data.keys())  # 데이터 내 key 확인

# Waveform 데이터 추출 및 시각화
waveform_sample = audio_data['waveform'][0]  # 첫 번째 샘플 사용 (키는 예시)
sample_rate = audio_data['sample_rate']  # 샘플 레이트 (있다면)

# Waveform 시각화
plt.figure(figsize=(12, 4))
plt.plot(waveform_sample.numpy())
plt.title("Audio MNIST Waveform Sample")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

## 데이터 전처리

## RNN 모델

## 손실 함수

## 모델 학습

## 학습 평가

## 예측 및 결과